<div style ="border: blue solid 5px; font-family: Agency FB;	"><center>
<a href="#" ></a> 
    <h1> <font color="blue">Traitement base ADEME</font> </h1> 
<a href="#" ></a>     
</center>
</div>

Dans ce notebook nous faisons les premiers traitements de la base ADEME. Ce traitement consiste à :
* Filtrer et retenir uniquement les unités de type "Entreprise"
* Enlever les siret et garder le premier Siren
* Enrichir la base par la création de la variable **évolution des émissions**

### Table des matières

* [Chargement de la base](#1-chargement-de-la-basetable-des-matières)
* [Filtre des entreprises](#2-filtre-des-entreprises-organisationtype)
* [Filtre des Siren](#3-filtre-des-siren)
* [Enrichissement](#4-enrichissement)
* [Sauvegarde](#5-sauvegarde-de-la-base)

## 1. Chargement de la base

In [5]:
import pandas as pd
import numpy as np
import os 

In [6]:
## Répertoire de travail
os.chdir("D:/01_ENSAI/Sem_2/Projet_Statistique/projet-risque-de-transition")

In [7]:
#Chargement du DF assessment
assessment = pd.read_excel('bdd/Ademe/bilans-ges.xlsx', sheet_name = 'assessments')
assessment

,id,organization_name,organization_description,organization_type,collectivity_type,staff,population,consolidation_method,reporting_year,total_scope_1,total_scope_2,total_scope_3,reference_year,action_plan,reductions_scope_1_2,reductions_scope_1,reductions_scope_2,reductions_scope_3,is_draft,source_url
0,1.0,Mutuelle d’Assurance des Instituteurs de France,Mutuelle d’assurance,Entreprise,NaN,6848.0,NaN,Opérationnel,2011.0,6017.000000,2170.000000,59931.000000,NaN,Oui,NaN,NaN,NaN,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...
1,2.0,ADIDAS France,ADIDAS France est une entreprise de distributi...,Entreprise,NaN,768.0,NaN,Opérationnel,2011.0,786.190000,516.260000,1164.090000,NaN,Oui,NaN,NaN,NaN,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...
2,3.0,CROUS Nice-Toulon,Les activités gérées par le CROUS de Nice-Toul...,Etablissement public,NaN,261.0,NaN,Opérationnel,2011.0,824.000000,382.000000,0.000000,NaN,Oui,NaN,NaN,NaN,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...
3,4.0,Fort de France,NaN,Collectivité territoriale (dont EPCI),Communes,3000.0,94000.0,NaN,2010.0,19811.000000,0.000000,48674.000000,NaN,Oui,NaN,NaN,NaN,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...
4,5.0,concarneau Cornouaille Agglomération,NaN,Collectivité territoriale (dont EPCI),Communes,102.0,49000.0,NaN,2011.0,977.160000,113.100000,0.000000,NaN,Non,NaN,NaN,NaN,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3871,7578.0,Laiterie Nouvelle de L'Arguenon,NaN,Entreprise,NaN,560.0,NaN,Opérationnel,2019.0,24596.000000,1454.000000,0.000000,NaN,Oui,NaN,2230.0,44.0,0.0,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...
3872,7581.0,NEO-SOFT SERVICES,ESN (Entreprise du Service Numérique) Néo-Soft...,Entreprise,NaN,1148.0,NaN,Opérationnel,2020.0,0.000000,8.950335,5763.819315,NaN,Oui,NaN,0.0,1.0,1000.0,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...
3873,7584.0,Fondation Santé Service,Activités hospitalières (8610Z),Association,NaN,1250.0,NaN,Opérationnel,2020.0,2086.178573,35.390441,764.399579,NaN,Oui,400.0,NaN,NaN,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...
3874,7588.0,Ciblex Express,Expert du transport express et de la livraison...,Entreprise,NaN,612.0,NaN,Opérationnel,2020.0,439.000000,110.000000,57004.000000,NaN,Oui,NaN,0.0,0.0,4560.0,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...


## 2. Filtre des entreprises (organisation_type)

La base contient des informations sur différents type d'organisation. 

Nous nous limiterons aux **"Entreprises"**

In [8]:
# Type d'institution présente dans la base
assessment.organization_type.value_counts()

Entreprise                               2461
Etablissement public                      866
Collectivité territoriale (dont EPCI)     317
Association                               118
Etat                                      114
Name: organization_type, dtype: int64

In [9]:
# Filtre des entreprises
assessment = assessment.drop(assessment[assessment.organization_type != "Entreprise"].index)

print("Finalement la base contient ",assessment.shape[0], " entreprises.")


Finalement la base contient  2461  entreprises.


## 3. Filtre des SIREN

In [10]:
# importation de la base legal_units
legal_units = pd.read_excel("bdd/Ademe/bilans-ges.xlsx", sheet_name = "legal_units")
legal_units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14278 entries, 0 to 14277
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   assessment_id       14278 non-null  float64
 1   legal_unit_id_type  14278 non-null  object 
 2   legal_unit_id       14278 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 334.8+ KB


In [11]:
# garder la liste de SIREN
print("Nombre de lignes de départ: ", len(legal_units), "\n")

legal_units = legal_units.drop(legal_units[legal_units.legal_unit_id_type == "SIRET"].index)
legal_units.drop_duplicates(subset= "assessment_id", keep = 'first', inplace = True, ignore_index=True)

print("Nombre de lignes apres suppression: ", len(legal_units), "\n")


Nombre de lignes de départ:  14278 

Nombre de lignes apres suppression:  3547 



**Fusion feuilles assessment et legal units**

In [12]:
bilan_ges = assessment.merge(legal_units, how = "left",left_on='id', right_on='assessment_id')

In [13]:
bilan_ges.sort_values(['legal_unit_id', 'reporting_year'], ascending=True, inplace=True)


**Qualité de la jointure**

In [14]:
## Taux de perdition
taux_perdition_0 = np.sum(bilan_ges['legal_unit_id'].isna())/bilan_ges.shape[0]*100
print("Apres fusion on obtient un taux de perdition de ", taux_perdition_0, "%")


Apres fusion on obtient un taux de perdition de  0.0 %


In [15]:
## Liste des SIREN dupliqué
dup_Siren = list(bilan_ges['legal_unit_id'].value_counts()[bilan_ges['legal_unit_id'].value_counts()>1].index)
## Liste des variables pour décrire ces unités dupliquées
list_dup = ['legal_unit_id', 'reporting_year', 'organization_name','total_scope_1', 'total_scope_2', 'staff']
## Observation des entités
bilan_ges.sort_values(by = ["legal_unit_id", "reporting_year"], ascending=False, inplace=True)
## Nom en miniscule
bilan_ges.organization_name = bilan_ges.organization_name.str.lower()
bilan_ges.organization_name = bilan_ges.organization_name.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

bilan_ges.loc[bilan_ges['legal_unit_id'].isin(dup_Siren), list_dup].head(20)


,legal_unit_id,reporting_year,organization_name,total_scope_1,total_scope_2,staff
1954,985520154,2019.0,les derives resiniques et terpeniques,22205.688000,3070.068000,749.0
345,985520154,2015.0,les derives resiniques et terpeniques,18591.639000,3533.343000,591.0
1193,975620675,2018.0,cosmetique active production,207.000000,0.100000,587.0
1003,975620675,2012.0,cosmetique active production,2432.000000,1453.000000,558.0
607,973510019,2015.0,delta security solutions,4358.600000,141.000000,1106.0
151,973510019,2014.0,delta security solutions,4775.000000,136.000000,1125.0
2067,973505357,2018.0,fedex express fr,37383.710000,1231.000000,6000.0
376,973505357,2014.0,tnt express france,2163.160000,56.160000,4694.0
978,971506191,2017.0,clauger,3945.600000,59.500000,1200.0
709,971506191,2016.0,clauger,503.769000,38.361000,500.0


## 4. Enrichissement

**Evolution émissions**

In [16]:
bilan_ges.sort_values(by = ["legal_unit_id", "reporting_year"], ascending=False, inplace=True)
count_siren = bilan_ges.groupby(["legal_unit_id"])["id"].count().reset_index(name="siren_count")
bilan_ges = bilan_ges.merge(count_siren, how = "left", left_on='legal_unit_id', right_on='legal_unit_id', validate='m:1')

In [17]:
df_group = bilan_ges.loc[bilan_ges['siren_count']>1,].groupby('legal_unit_id').head(2)
df_group.head(15)

,id,organization_name,organization_description,organization_type,collectivity_type,staff,population,consolidation_method,reporting_year,total_scope_1,...,reductions_scope_1_2,reductions_scope_1,reductions_scope_2,reductions_scope_3,is_draft,source_url,assessment_id,legal_unit_id_type,legal_unit_id,siren_count
2,5633.0,les derives resiniques et terpeniques,Fabrication de produits chimiques organiques.,Entreprise,NaN,749.0,NaN,Opérationnel,2019.0,22205.688000,...,NaN,3261.0,461.0,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...,5633.0,SIREN,985520154,2
3,830.0,les derives resiniques et terpeniques,Fabrication de produits chimiques organiques,Entreprise,NaN,591.0,NaN,Opérationnel,2015.0,18591.639000,...,780.0,NaN,NaN,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...,830.0,SIREN,985520154,2
5,3889.0,cosmetique active production,Fabrication et conditionnement de produits cos...,Entreprise,NaN,587.0,NaN,Opérationnel,2018.0,207.000000,...,124.0,NaN,NaN,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...,3889.0,SIREN,975620675,2
6,3223.0,cosmetique active production,fabrication et conditionnement de produits cos...,Entreprise,NaN,558.0,NaN,Opérationnel,2012.0,2432.000000,...,NaN,186.0,24.0,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...,3223.0,SIREN,975620675,2
7,1761.0,delta security solutions,installation maintenance et télésurveillance d...,Entreprise,NaN,1106.0,NaN,Opérationnel,2015.0,4358.600000,...,NaN,1281.0,0.0,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...,1761.0,SIREN,973510019,2
8,387.0,delta security solutions,Installation et maintenance de systèmes de séc...,Entreprise,NaN,1125.0,NaN,Opérationnel,2014.0,4775.000000,...,NaN,294.0,NaN,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...,387.0,SIREN,973510019,2
9,5917.0,fedex express fr,Transport de marchandises express en France et...,Entreprise,NaN,6000.0,NaN,Opérationnel,2018.0,37383.710000,...,2420.0,NaN,NaN,951.0,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...,5917.0,SIREN,973505357,2
10,906.0,tnt express france,"En France, TNT est le leader sur le marché de...",Entreprise,NaN,4694.0,NaN,Opérationnel,2014.0,2163.160000,...,3334.0,NaN,NaN,4893.0,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...,906.0,SIREN,973505357,2
11,3157.0,clauger,Leader en froid industriel et traitement d'air...,Entreprise,NaN,1200.0,NaN,Opérationnel,2017.0,3945.600000,...,NaN,350.0,2.0,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...,3157.0,SIREN,971506191,2
12,2117.0,clauger,Concepteur de solution dans le froid industrie...,Entreprise,NaN,500.0,NaN,Opérationnel,2016.0,503.769000,...,NaN,450.0,35.0,NaN,Non,http://www.bilans-ges.ademe.fr/fr/bilanenligne...,2117.0,SIREN,971506191,2


In [18]:

# df_group: base avec les établissements ayant au moins deux reporting
df_group = bilan_ges.loc[bilan_ges['siren_count']>1,].groupby('legal_unit_id').head(2)
df_group['evolution_scope_1'] = [i for i in range(df_group.shape[0])]
for i in range(df_group.shape[0] - 1):
   df_group.loc[df_group.index[i], 'evolution_scope_1'] = (df_group.loc[df_group.index[i],'total_scope_1'] - df_group.loc[df_group.index[i+1],'total_scope_1'])

df_group[['legal_unit_id', 'siren_count', 'total_scope_1', 'evolution_scope_1']]
df_group.drop_duplicates(subset= "legal_unit_id", keep = 'first', inplace = True, ignore_index=True)

## Recupération de la variable evolution scope 1
bilan_ges = bilan_ges.merge(df_group[['legal_unit_id', 'evolution_scope_1']], how='left', on = 'legal_unit_id', validate= 'm:1')

In [19]:
print(bilan_ges.evolution_scope_1.describe())
print("Taux Na : ", round(bilan_ges.evolution_scope_1.isna().sum()/bilan_ges.shape[0]*100, 3), "%")


count    1.041000e+03
mean    -3.594655e+03
std      6.721407e+04
min     -1.081137e+06
25%     -9.980000e+02
50%     -3.300000e+01
75%      6.230000e+02
max      3.177380e+05
Name: evolution_scope_1, dtype: float64
Taux Na :  57.7 %


In [20]:
Ademe = bilan_ges.drop_duplicates(subset = "legal_unit_id", keep = 'first', ignore_index=True)
print("On passe donc de 2461 à ", Ademe.shape[0], " entreprises")

On passe donc de 2461 à  1860  entreprises


**Suppression des variables inutiles**

In [21]:
list_var = [
            "organization_description",
            "organization_type", 
            "collectivity_type", 
            "consolidation_method", 
            "legal_unit_id_type", 
            "source_url", 
            "population", 
            "is_draft",
            "reductions_scope_1",
            "reductions_scope_2",
            "reductions_scope_3",
            "reductions_scope_1_2",
            "reference_year",
            'siren_count'
            ]
Ademe.drop(list_var, axis= 1, inplace=True)

C:\Users\papet\AppData\Local\Temp\ipykernel_13720\851812775.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ademe.drop(list_var, axis= 1, inplace=True)


In [22]:
list_var = ["assessment_id", 
            "organization_description",
            "organization_type", 
            "collectivity_type", 
            "consolidation_method", 
            "legal_unit_id_type", 
            "source_url", 
            "population", 
            "is_draft",
            "reductions_scope_1",
            "reductions_scope_2",
            "reductions_scope_3",
            "reductions_scope_1_2",
            "reference_year",
            'siren_count'
            ]
bilan_ges.drop(list_var, axis= 1, inplace=True)

**Taux de valeur manquante**

In [23]:
taux_na = Ademe.isna().sum()/Ademe.shape[0]*100
taux_na.values
var =list(Ademe.columns)
dtype = Ademe.dtypes.values
data = {
         'Variables' : var,
         'Dtype': dtype,
         'Taux_na': taux_na.values    

      }
recap_na = pd.DataFrame(data)
recap_na.sort_values(by = 'Taux_na', ascending=False, inplace= True)
recap_na


,Variables,Dtype,Taux_na
10,evolution_scope_1,float64,74.086022
4,total_scope_1,float64,1.129032
5,total_scope_2,float64,1.129032
6,total_scope_3,float64,1.129032
0,id,float64,0.000000
1,organization_name,object,0.000000
2,staff,float64,0.000000
3,reporting_year,float64,0.000000
7,action_plan,object,0.000000
8,assessment_id,float64,0.000000


## 5. Sauvegarde de la base

In [24]:
Ademe.to_csv('bdd/Ademe/Ademe.csv', index= False)
## Base contenant les informations pour plusieurs reporting
bilan_ges.to_csv("bdd/Ademe/Ademe_dupli_siren.csv", index = False)